In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from statsmodels.stats.multitest import multipletests

In [2]:


all_mean_arr=0.03411387460093794
all_std_arr=0.01334116014092687
mean_dict_different_race={'ACB': 0.046621686769679045,
 'ASW': 0.0465892685112964,
 'BEB': 0.035970363185117774,
 'CDX': 0.03266259792789275,
 'CEU': 0.03488434818325704,
 'CHB': 0.03285897897059889,
 'CHS': 0.032796236687928325,
 'CLM': 0.036503986698316906,
 'ESN': 0.04643492431530403,
 'FIN': 0.034935842703769975,
 'GBR': 0.03498406394197642,
 'GIH': 0.035593525252319486,
 'GWD': 0.045836194805001416,
 'IBS': 0.035140910353782764,
 'ITU': 0.0347164844670136,
 'JPT': 0.03273063391657775,
 'KHV': 0.033023382087129426,
 'LWK': 0.0466350005070252,
 'MSL': 0.047257776857587634,
 'MXL': 0.03529323182476227,
 'PEL': 0.03222809053641309,
 'PJL': 0.03450825905794522,
 'PUR': 0.037936602232609126,
 'STU': 0.03454596363552346,
 'TSI': 0.03508890911947475,
 'YRI': 0.04649868546958651}
std_dict_different_race={'ACB': 0.01660837078934563,
 'ASW': 0.016852430054156438,
 'BEB': 0.016339814570213686,
 'CDX': 0.01657929168371163,
 'CEU': 0.01615285680476383,
 'CHB': 0.0167769938917591,
 'CHS': 0.016361466363591634,
 'CLM': 0.016076712212741314,
 'ESN': 0.016439364809682746,
 'FIN': 0.016395558138330903,
 'GBR': 0.01654186443581865,
 'GIH': 0.016487954511321803,
 'GWD': 0.01637735033664222,
 'IBS': 0.01625740426618882,
 'ITU': 0.015999114837619923,
 'JPT': 0.016777856667394144,
 'KHV': 0.016805860157616894,
 'LWK': 0.01656542896110204,
 'MSL': 0.0168657687428544,
 'MXL': 0.01631757749279766,
 'PEL': 0.015903810806753623,
 'PJL': 0.015785059411911464,
 'PUR': 0.016016827217755162,
 'STU': 0.01615897181252148,
 'TSI': 0.016499539576161064,
 'YRI': 0.01656520483641468}

In [3]:
# split_all_region={'split_chr14_xaa': [[19806333, 19976578]],
#  'split_chr7_xbc': [[124810550, 125070920]],
#  'split_chr1_xak': [[45762038, 46100296]],
#  'split_chr1_xbk': [[188243362, 188513984]]}

In [2]:
#mhc互补区域
mhc_race_positions_dict={'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
'All':{'mhc':[[29720403, 29913914], [30994370, 31528792], [32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [29939668, 30154438],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32423532, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'GWD': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'IBS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ITU': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'JPT': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'KHV': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'LWK': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32397207, 32923168]]},
 'MSL': {'mhc': [[29720403, 30011739],
   [30959575, 31528792],
   [32212726, 32882258]]},
 'MXL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'PEL': {'mhc': [[29720403, 29913914],
   [30959575, 31528792],
   [32288923, 32882258]]},
 'PJL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'PUR': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'STU': {'mhc': [[29720403, 29913914],
   [30994370, 31577278],
   [32212726, 32882258]]},
 'TSI': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'YRI': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]}}

In [3]:
split_race_region={'ACB': {'chr3': [[98029388, 98331063]],
  'chr4': [[8996392, 9139446]],
  'chr5': [[34277945, 34574089]]},
 'ASW': {'chr1': [[248297800, 248584618]]},
 'BEB': {'chr13': [[55108214, 55447878]]},
 'CDX': {'chr7': [[64033450, 64433853]], 'chr15': [[24061424, 24264850]]},
 'CEU': {'chr17': [[45825257, 46352295]]},
 'CHB': {},
 'CHS': {},
 'CLM': {'chr3': [[95042564, 95345546],
   [98029388, 98261301],
   [163846034, 164208695]],
  'chr17': [[45825257, 46352295]]},
 'ESN': {'chr1': [[248297800, 248584618]], 'chr5': [[34277945, 34574089]]},
 'FIN': {},
 'GBR': {'chr1': [[227003796, 227338558]],
  'chr3': [[98029388, 98261301]],
  'chr17': [[45825257, 46352295]]},
 'GIH': {'chr3': [[98029388, 98331063]], 'chr14': [[19806333, 20078222]]},
 'GWD': {'chr1': [[248297800, 248584618]],
  'chr5': [[34277945, 34574089]],
  'chr7': [[11926418, 12230106]]},
 'IBS': {'chr1': [[227003796, 227338558]], 'chr17': [[45825257, 46352295]]},
 'ITU': {'chr17': [[52570659, 52892893]]},
 'JPT': {},
 'KHV': {},
 'LWK': {'chr4': [[8996392, 9139446]]},
 'MSL': {'chr5': [[34231981, 34574089]]},
 'MXL': {'chr1': [[227003796, 227338558]], 'chr11': [[124162560, 124482002]]},
 'PEL': {'chr3': [[98029388, 98331063]], 'chr11': [[23181031, 23574546]]},
 'PJL': {'chr3': [[163846034, 164208695]]},
 'PUR': {'chr3': [[98029388, 98331063]], 'chr17': [[45825257, 46352295]]},
 'STU': {'chr3': [[98029388, 98293778]], 'chr13': [[55108214, 55447878]]},
 'TSI': {'chr7': [[67111221, 67437767]], 'chr17': [[45825257, 46352295]]},
 'YRI': {'chr1': [[227003796, 227338558], [248297800, 248584618]],
  'chr3': [[166091993, 166408110]],
  'chr5': [[34277945, 34574089]]}}

In [6]:
import re
def getinteger(s):
    match = re.search(r'chr(\d+)', s)

    if match:
        return match.group(1) 


In [7]:
df_mhc_all=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/csvfiles/mhc.csv")
df_mhc_race=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/race/posmhc.csv")


In [8]:
allrecords=[]
for race,values in mhc_race_positions_dict.items():
    if values!={}:
        if race=="All":
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=["All",6,1]
                top_1000_P_list=df_mhc_all[(df_mhc_all["Unnamed: 0"]>=region[0])&(df_mhc_all["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)
        else:
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=[race,6,1]
                top_1000_P_list=df_mhc_race[(df_mhc_race["Unnamed: 0"]>=region[0])&(df_mhc_race["Unnamed: 0"]<=region[1])][race].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, mean_dict_different_race[race], std_dict_different_race[race])),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)


In [9]:
for race,values in split_race_region.items():
    # if race!="All":
    #     # for k, v in values.items():
    #     #     chr=getinteger(k.split("_")[1])
    #     #     filename="pos_prob_"+k+".csv"
    #     #     df=pd.read_csv("../csvfiles/"+filename)
    #     #     for region in v:
    #     #         onerecord=[]
    #     #         onerecord+=["All",chr,0]
    #     #         top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])]["P"].to_list()
    #     #         # sortedlist=sorted(top_1000_P_list,reverse=True)
    #     #         # top_1000=sortedlist[:1000]
    #     #         mean_1000_P=np.mean(top_1000_P_list)
    #     #         std_1000_P=np.std(top_1000_P_list, ddof=1)
    #     #         pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
    #     #         onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
    #     #         allrecords.append(onerecord)
    #     continue
    # else:
    for k,v in values.items():
        chr=getinteger(k)
        filename=k+".csv"
        df=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/race_integration/"+filename)
        for region in v:
            onerecord=[]
            onerecord+=[race,chr,0]
            top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])][race].to_list()
            # sortedlist=sorted(top_1000_P_list,reverse=True)
            # top_1000=sortedlist[:1000]
            mean_1000_P=np.mean(top_1000_P_list)
            std_1000_P=np.std(top_1000_P_list, ddof=1)
            pvalue=round((1-stats.norm.cdf(mean_1000_P,mean_dict_different_race[race], std_dict_different_race[race])),15)
            onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
            allrecords.append(onerecord)

In [10]:
#csv1
columns=["Race","chr","isMHC","Average of P","Std of P","P-value","start","end","len"]

In [11]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    # dataframe(dictforDF).to_csv("../formalcsvs/csv1_2023_05_04.csv")
    return dataframe(dictforDF)





In [12]:
dfnew=generatedf(columns=columns,allrecords=allrecords)

In [13]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len
0,ACB,6,1,0.170126,0.198950,5.200000e-14,29720403,30011739,291336
1,ACB,6,1,0.122783,0.161446,2.262400e-06,30994370,31528792,534422
2,ACB,6,1,0.179201,0.210767,1.000000e-15,32212726,32882258,669532
3,All,6,1,0.133014,0.155387,6.200000e-14,29720403,29913914,193511
4,All,6,1,0.130397,0.156856,2.660000e-13,30994370,31528792,534422
...,...,...,...,...,...,...,...,...,...
117,TSI,17,0,0.120254,0.220970,1.223791e-07,45825257,46352295,527038
118,YRI,1,0,0.085054,0.171443,9.970180e-03,227003796,227338558,334762
119,YRI,1,0,0.100586,0.207070,5.470850e-04,248297800,248584618,286818
120,YRI,3,0,0.084466,0.152598,1.095336e-02,166091993,166408110,316117


In [14]:

pvalue_list=dfnew["P-value"].to_list()
pvalue_list

[5.2e-14,
 2.262400474e-06,
 1e-15,
 6.2e-14,
 2.66e-13,
 0.0,
 5e-15,
 1.47609e-10,
 2.8e-14,
 0.0,
 5.224091e-09,
 1.57e-13,
 6.809e-12,
 1.47e-13,
 2.04153e-10,
 0.0,
 4.5915e-11,
 1.194554e-08,
 1.36534e-10,
 8e-15,
 1.3734894187e-05,
 3.4897928e-08,
 0.0,
 2.3910107e-08,
 3.045e-12,
 0.0,
 1.1e-14,
 1.9603232e-08,
 4.53e-13,
 2.53e-13,
 4.988e-12,
 3.2424498e-08,
 0.0,
 1.446421063e-06,
 2.377e-12,
 0.0,
 0.0,
 1.8744795e-08,
 3.852e-12,
 9.7070687e-08,
 5.343e-12,
 0.0,
 1.16647129e-07,
 8.74053e-10,
 0.0,
 1.56e-13,
 9.709e-12,
 5.8055597e-08,
 0.0,
 9e-15,
 7.45862e-10,
 0.0,
 5.3e-14,
 1.2150829e-08,
 1.034e-12,
 0.0,
 1.5089772e-08,
 9.8e-14,
 7.381e-12,
 3.0549e-11,
 0.0,
 1.671593056e-06,
 1.831e-12,
 1e-15,
 9.76249465e-07,
 2.239e-11,
 2.3e-14,
 2.25e-13,
 1.251e-12,
 1e-15,
 1.427e-12,
 8.006e-12,
 0.0,
 3.64939657e-07,
 8.859305221e-06,
 4.228298e-09,
 0.0,
 0.0,
 2.5039743e-08,
 8e-15,
 5.8135663225e-05,
 0.014158546237545,
 0.001501028880938,
 0.000756914092605,
 0.00

In [15]:
pvalue_list=dfnew["P-value"].to_list()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
corr_pvals

array([2.22965517e-13, 3.28586736e-06, 6.10000000e-15, 2.52133333e-13,
       8.77081081e-13, 0.00000000e+00, 2.90476190e-14, 3.21576750e-10,
       1.26518519e-13, 0.00000000e+00, 1.02796629e-08, 5.63352941e-13,
       1.73062083e-11, 5.60437500e-13, 4.29425276e-10, 0.00000000e+00,
       1.03733889e-10, 2.31326330e-08, 3.02857236e-10, 4.24347826e-14,
       1.92604263e-05, 5.99654537e-08, 0.00000000e+00, 4.28975449e-08,
       8.44295455e-12, 0.00000000e+00, 5.36800000e-14, 3.56954374e-08,
       1.45436842e-12, 8.57388889e-13, 1.32290435e-11, 5.65112679e-08,
       0.00000000e+00, 2.17856012e-06, 6.74404651e-12, 0.00000000e+00,
       0.00000000e+00, 3.46494695e-08, 1.04432000e-11, 1.60035457e-07,
       1.38690638e-11, 0.00000000e+00, 1.89745997e-07, 1.77724110e-09,
       0.00000000e+00, 5.63352941e-13, 2.32254510e-11, 9.83719838e-08,
       0.00000000e+00, 4.57500000e-14, 1.54229092e-09, 0.00000000e+00,
       2.22965517e-13, 2.31625178e-08, 3.23456410e-12, 0.00000000e+00,
      

In [16]:
dfnew = dfnew.assign(corr_pvals=corr_pvals)


In [17]:
dfnew.to_csv("tables1_dis.csv")